In [1]:
import numpy as np
from collections import OrderedDict as odict
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

This notebook computes the latencies and bandwidths of the three primitive function types

In [2]:
#(hardware name, number of nodes)
filesD = {
         'knl_mpi1':('knl',1), 'knl_mpi2':('knl',2), 'knl_mpi4':('knl',4),
         'skl_mpi1':('skl',1), 'skl_mpi2':('skl',2), 'skl_mpi4':('skl',4),
         'i5':('i5',1),
         'p100_mpi1':('p100',1), 'p100_mpi2':('p100',2), 'p100_mpi4':('p100',4),
         'v100_mpi1':('v100',1), 'v100_mpi2':('v100',2), 'v100_mpi4':('v100',4),
         'gtx1060':('gtx1060',1)
        }

files = odict(sorted(filesD.items(), key= lambda t : t[1][1]))
pd.set_option('precision',1)

#### Axpby and Dot Latencies
The latencies are determined by taking the minimum of the average runtimes 

#### Axpby and Dot Bandwidths
The bandwidths are determined by taking the average bandwidth of the 30 bandwidths corresponding to the 3 largest sizes.

#### Dx-Dy Latencies
As in Axpby 

#### Dx-Dy Bandwidths
Since the efficiency of the matrix-vector multiplications depends on the polynomial coefficient we should compute these bandwidths separately


In [3]:
names={'axpby':3,'dot':2,'dx':3, 'dy':3}
#ns=[3,4]
values = []
for f, v in files.items() :#{'knl_mpi2':('knl',2)}.items():
    runtimes=pd.read_csv('benchmark_'+f+'.csv', delimiter=' ')
    #add size and bandwidth columns
    runtimes.insert(0,'size', 8*runtimes['n']*runtimes['n']
                    *runtimes['Nx']*runtimes['Ny']/1e6/v[1]) #inplace transformation
    for name,memops in names.items() :
        runtimes.insert(0,name+'_bw',runtimes['size']/1000*memops/runtimes[name])
    runtimes = runtimes.assign( dxdy=(runtimes['dx']+runtimes['dy'])/2)
    runtimes = runtimes.assign( dxdy_bw=2.0*runtimes['dx_bw']*runtimes['dy_bw']
                               /(runtimes['dx_bw']+runtimes['dy_bw']))
    #compute one version with aggregated grouped sizes and one without
    avgruntimes=runtimes.groupby(['n', 'Nx','Ny','size']).agg(['mean', 'std'])
    avgruntimes=avgruntimes.reset_index(level=['n','Nx','Ny','size'])
    avgruntimes.sort_values(by='size',inplace=True) #sort by size
    runtimes.sort_values(by='size',inplace=True)
    ##first compute axpby and dot latencies and bandwidths 
    nmax = 3
    s =30

    line = []
    l=len(runtimes)
    line.append(v[0]) #0
    line.append(v[1]) #1
    for q in ['axpby','dot']:
        bandwidth       = runtimes[l-s:l][q+'_bw'].mean()
        err_bandwidth   = runtimes[l-s:l][q+'_bw'].std()
        mean_latency    = avgruntimes[0:nmax][(q,'mean')].mean()/1e-6
        min_latency     = avgruntimes[(q,'mean')].min()/1e-6
        idx_min_latency = avgruntimes[(q,'mean')].idxmin()
        corr_min_latency= min_latency - avgruntimes['size'].loc[idx_min_latency]*names[q]/bandwidth/1e-3
        if corr_min_latency <0 : corr_min_latency = 0 
        line.append(bandwidth) #2 bandwidth
        line.append(err_bandwidth)  #3 err_bandwidth
        line.append(mean_latency) #4 latency mean
        line.append(min_latency)  #5 latency min
        line.append(corr_min_latency ) #6 corrected lat 

    ##now compute latency and bandwidths of dx and y
    for n in [2,3,4,5]:
        #take n
        dxdy=runtimes[runtimes['n']==n]
        
        avgdxdy = avgruntimes[avgruntimes['n']==n]
        dxdy=dxdy.sort_values(by='size')
        avgdxdy=avgdxdy.sort_values(by='size') #sort by size
        bandwidth       = dxdy[(dxdy['size']>10)&(dxdy['size']<1000)]['dxdy_bw'].mean()
        err_bandwidth   = dxdy[(dxdy['size']>10)&(dxdy['size']<1000)]['dxdy_bw'].std()
        mean_latency    = avgdxdy[0:nmax][('dxdy','mean')].mean()/1e-6
        min_latency     = avgdxdy[('dxdy','mean')].min()/1e-6
        idx_min_latency = avgdxdy[('dxdy','mean')].idxmin()
        corr_min_latency= min_latency - avgdxdy['size'].loc[idx_min_latency]*names['dx']/bandwidth/1e-3
        if corr_min_latency <0 : corr_min_latency = 0 
        line.append(bandwidth) #2 bandwidth
        line.append(err_bandwidth)  #3 err_bandwidth
        line.append(mean_latency) #4 latency mean
        line.append(min_latency)  #5 latency min
        line.append(corr_min_latency ) #6 corrected lat        
    #print(line)    
    values.append(line)

In [4]:
#now construct new table with values from previous cell      
tuples=[('arch','',''),('nodes','','')]        
for q in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    tuples.append((q,'bw','avg'))
    tuples.append((q,'bw','std'))
    tuples.append((q,'lat','avg'))
    tuples.append((q,'lat','min'))
    tuples.append((q,'lat','bw'))

cols=pd.MultiIndex.from_tuples(tuples)
arr = pd.DataFrame(values,index=files.keys(), columns=cols)
arr.sort_values(by='arch',inplace=True)
arr.set_index(['arch','nodes'],inplace=True)
#arr.loc[:,[('dot','bw','avg'),('dot','lat','avg')]]
arr

axpby                              dot                         \
                  bw            lat                bw             lat          
                 avg      std   avg   min   bw    avg      std    avg    min   
arch    nodes                                                                  
gtx1060 1      157.0  5.6e-02  23.2   3.5  0.0   26.5  9.8e-02  199.9  131.6   
i5      1       30.0  1.9e-01  30.6  12.4  0.0    9.3  3.7e-02  316.7  117.4   
knl     1      393.2  2.2e+01  11.8  10.0  5.5  141.4  6.6e+00   77.1   63.2   
        2      420.4  4.0e+01  10.7  10.0  7.9  128.4  2.7e+00   99.4   91.5   
        4      420.7  4.4e+01  10.5  10.2  9.2  109.7  3.9e+00  126.1  122.3   
p100    1      553.3  6.9e-01   4.1   3.1  0.2  346.1  1.5e+00   54.9   51.2   
        2      554.2  9.8e-02   3.3   3.0  1.6  339.8  3.0e+00   52.9   49.5   
        4      555.0  2.1e-01   3.2   3.1  2.4  328.1  5.2e+00   54.4   49.0   
skl     1      206.7  5.9e+00   4.6   4.1  0.0  192.1  1.8e+01   32.3   24.2   
        2      216.3  7.0e+00   4.2   3.9  0.0  182.3  1.2e+01   31.3   25.9   
        4      232.6  1.5e+01   4.1   4.0  0.0  166.9  2.4e+01   43.8   42.4   
v100    1      848.4  6.7e-01   3.7   3.1  1.2  593.3  2.1e+00   36.8   35.7   
        2      846.9  4.3e+00   3.4   3.1  2.2  580.4  1.1e+01   38.8   37.7   
        4      845.0  1.9e+00   3.4   3.3  2.3  551.5  1.0e+01   38.6   35.2   

                      ...   dxdy4                             dxdy5        \
                      ...      bw           lat                  bw         
                  bw  ...     avg   std     avg    min    bw    avg   std   
arch    nodes         ...                                                   
gtx1060 1       92.1  ...    83.8  13.8   322.6   72.5   0.0   69.3  17.0   
i5      1        4.8  ...    25.6   1.5  1212.1  208.3   0.0   21.4   1.9   
knl     1       54.8  ...   126.0  18.6   215.7   52.1   0.0  101.3  14.9   
        2       86.9  ...   109.4  14.3   207.8   92.4  59.9   88.5   7.4   
        4      119.6  ...   115.7  21.7   155.1   82.9  67.5   89.2  13.8   
p100    1       48.2  ...   200.3   1.8   148.5   29.9   0.0  165.4  14.0   
        2       48.0  ...   185.3  12.5   125.3   61.9  44.9  159.4  11.0   
        4       48.2  ...   174.2  14.7   100.7   70.3  61.3  150.1  15.0   
skl     1       17.3  ...   118.1  18.4   340.9   82.8  15.4  110.9   8.0   
        2       22.2  ...   120.4  20.5   149.1   63.8  30.7  114.3   7.2   
        4       37.9  ...   114.3  24.6    91.0   47.4  30.0  110.4   9.2   
v100    1       34.0  ...   649.4  15.4    46.9   12.1   2.4  535.5  48.8   
        2       36.8  ...   535.2  81.8    71.3   52.3  46.4  462.8  74.6   
        4       34.7  ...   467.8  80.0    78.4   70.6  67.2  407.7  83.4   

                                    
                  lat               
                  avg    min    bw  
arch    nodes                       
gtx1060 1       571.4  125.0   0.0  
i5      1      1956.6  340.4   0.0  
knl     1       451.5   98.3   0.0  
        2       363.1  133.9  71.2  
        4       235.9  108.1  77.0  
p100    1       273.0   62.6   3.1  
        2       188.5   75.5  44.6  
        4       134.2   74.5  58.1  
skl     1       442.7  132.5  20.3  
        2       249.6   89.1  34.7  
        4       142.5   61.7  33.6  
v100    1        85.4   23.5   5.2  
        2        92.7   58.2  47.6  
        4        89.4   71.7  65.6  

[14 rows x 30 columns]

In [5]:
#arr=arr.reset_index()

In [6]:
#define conversion function 
def toString(x): 
    if pd.isnull(x) : return 'n/a'
    #string = '%.1f'% x
    string = '%d' %np.ceil(x)
    #if np.ceil(x)<100 : string = '0'+string
    if np.ceil(x)<10 : string = '0'+string
    return string

In [7]:
addto = []
for n in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    arr.loc[:,(n,'bw','string')]= arr[n]['bw']['avg'].apply(toString) +" ± "+arr[n]['bw']['std'].apply(toString)
    addto.append((n,'lat','bw'))
    addto.append((n,'bw','string'))

#make a table for display
nicetable=arr[addto]
drop = nicetable.columns.droplevel(2)
nicetable.columns=drop
#nicetable.reset_index(inplace=True)
#nicetable.set_index('arch')
newindex=[('i5',1)]
for n in ['skl','knl']:
    for m in [1,2,4]:
        newindex.append((n,m))
newindex.append(('gtx1060',1))
for n in ['p100','v100']:
    for m in [1,2,4]:
        newindex.append((n,m))
    
nicetable=nicetable.reindex(newindex)

nicetable

axpby              dot           dxdy2           dxdy3  \
                lat        bw    lat        bw   lat        bw   lat   
arch    nodes                                                          
i5      1       0.0   30 ± 01    4.8   10 ± 01   0.0   28 ± 03   0.0   
skl     1       0.0  207 ± 06   17.3  193 ± 19  22.7  182 ± 36  16.3   
        2       0.0  217 ± 08   22.2  183 ± 12  29.4  175 ± 45  30.3   
        4       0.0  233 ± 16   37.9  167 ± 24  28.5  168 ± 44  29.7   
knl     1       5.5  394 ± 23   54.8  142 ± 07   9.9  240 ± 18   7.7   
        2       7.9  421 ± 41   86.9  129 ± 03  49.0  176 ± 28  55.8   
        4       9.2  421 ± 44  119.6  110 ± 04  52.7  156 ± 25  58.9   
gtx1060 1       0.0  158 ± 01   92.1   27 ± 01   0.0  131 ± 01   2.7   
p100    1       0.2  554 ± 01   48.2  347 ± 02   1.8  288 ± 03   0.6   
        2       1.6  555 ± 01   48.0  340 ± 03  46.0  249 ± 20  43.9   
        4       2.4  555 ± 01   48.2  329 ± 06  63.5  228 ± 16  61.5   
v100    1       1.2  849 ± 01   34.0  594 ± 03   3.1  802 ± 17   2.7   
        2       2.2  847 ± 05   36.8  581 ± 11  49.6  575 ± 94  48.3   
        4       2.3  845 ± 02   34.7  552 ± 11  66.6  469 ± 57  66.2   

                         dxdy4           dxdy5            
                      bw   lat        bw   lat        bw  
arch    nodes                                             
i5      1        30 ± 03   0.0   26 ± 02   0.0   22 ± 02  
skl     1       162 ± 13  15.4  119 ± 19  20.3  111 ± 09  
        2       158 ± 17  30.7  121 ± 21  34.7  115 ± 08  
        4       160 ± 08  30.0  115 ± 25  33.6  111 ± 10  
knl     1       173 ± 27   0.0  127 ± 19   0.0  102 ± 15  
        2       142 ± 23  59.9  110 ± 15  71.2   89 ± 08  
        4       128 ± 24  67.5  116 ± 22  77.0   90 ± 14  
gtx1060 1       112 ± 02   0.0   84 ± 14   0.0   70 ± 18  
p100    1       238 ± 04   0.0  201 ± 02   3.1  166 ± 15  
        2       217 ± 14  44.9  186 ± 13  44.6  160 ± 12  
        4       193 ± 24  61.3  175 ± 15  58.1  151 ± 16  
v100    1       713 ± 20   2.4  650 ± 16   5.2  536 ± 49  
        2       576 ± 81  46.4  536 ± 82  47.6  463 ± 75  
        4      438 ± 115  67.2  468 ± 80  65.6  408 ± 84

#### Assumptions
- there are three basic functions: trivially parallel(axpby), nearest neighbor (dxdy), global reduction (dot)
- each can be represented by the single node bandwidth, the single node latency and the multinode latency

#### But
- does not capture cache effect e.g. in SKl

In [8]:
index = ['i5','skl','knl','gtx1060','p100','v100']  
lines = []
for arch in  index: 
    line = []
    line.append(arch)
    #first the bandwidths
    line.append( arr.loc[(arch,1),('axpby','bw','avg')] )
    for n in ['dot','dxdy2','dxdy3','dxdy4','dxdy5']:
        line.append( arr.loc[(arch,1),(n,'bw','avg')] /line[1])
    for n in ['axpby','dot','dxdy2'] :
        line.append( arr.loc[(arch,1),(n,'lat','bw')] )
        if arch == 'i5' or arch == 'gtx1060':
            line.append(None)
        else:
            line.append( arr.loc[(arch,4),(n,'lat','bw')] )
    lines.append(line)
    
tuples=['arch']     

for n in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    tuples.append(n+'_bw')
for n in ['axpby','dot','dxdy']:
    tuples.append(n+'_lat_shared')
    tuples.append(n+'_lat_dist')
cols=tuples
toDisk = pd.DataFrame(lines, columns=cols)
toDisk.to_csv('performance.csv',sep=' ',index=False)

In [9]:
pd.set_option('precision',2)
test = pd.read_csv('performance.csv',delimiter=' ')
test

,arch,axpby_bw,dot_bw,dxdy2_bw,dxdy3_bw,dxdy4_bw,dxdy5_bw,axpby_lat_shared,axpby_lat_dist,dot_lat_shared,dot_lat_dist,dxdy_lat_shared,dxdy_lat_dist
0,i5,29.99,0.31,0.93,0.97,0.85,0.72,0.00,NaN,4.76,NaN,0.00,NaN
1,skl,206.71,0.93,0.88,0.78,0.57,0.54,0.00,0.00,17.28,37.93,22.70,28.52
2,knl,393.15,0.36,0.61,0.44,0.32,0.26,5.47,9.16,54.83,119.59,9.93,52.67
3,gtx1060,157.05,0.17,0.83,0.71,0.53,0.44,0.00,NaN,92.06,NaN,0.00,NaN
4,p100,553.33,0.63,0.52,0.43,0.36,0.30,0.23,2.42,48.15,48.16,1.82,63.51
5,v100,848.36,0.70,0.95,0.84,0.77,0.63,1.22,2.30,33.95,34.73,3.11,66.63


#### Observations
- note the high latency in the knl MPI implementation of dxdy. It seems to suffer from the same problem as the GPUs. (Is this the speed of PCIe we see?)

In [10]:
index = ['i5','skl','knl','gtx1060','p100','v100']  
#theo = [38,None,None,192,732,898]
lines = []
for arch in  index: 
    line = []
    #line.append(arch)
    #first the bandwidths
    base_bw = arr.loc[(arch,1), ('axpby','bw','avg')]
    err_bw = arr.loc[(arch,1), ('axpby','bw','std')]
    
    line.append( toString(base_bw)+" $\pm$ "+toString(err_bw) )
    line.append( toString(arr.loc[(arch,1),('axpby','lat','bw')]) )
    if arch == 'i5' or arch == 'gtx1060':
        line.append(toString(None))
    else:
        line.append( toString(arr.loc[(arch,4),('axpby','lat','bw')]) )
        
    for n in ['dot','dxdy2','dxdy3','dxdy4','dxdy5']:
        new_bw = arr.loc[(arch,1),(n,'bw','avg')]
        new_err = arr.loc[(arch,1),(n,'bw','std')]
        #efficiency
        #line.append( toString( new_bw/base_bw*100)+" $\pm$ "+toString(100*(err_bw/new_bw+new_bw/base_bw/base_bw*new_err))) 
        #bandwidth
        line.append( toString( new_bw)+" $\pm$ "+toString(new_err))
        line.append( toString(arr.loc[(arch,1),(n,'lat','bw')]) )
        if arch == 'i5' or arch == 'gtx1060':
            line.append(toString(None))
        else:
            if n == 'dot':
                line.append( toString(arr.loc[(arch,4),(n,'lat','bw')]) )
            else:
                line.append(toString(arr.loc[(arch,4),('dxdy2','lat','bw')]))
                
    lines.append(line)
    
tuples=[]  


for p in ['axpby','dot','dxdy (P=2)','dxdy (P=3)','dxdy (P=4)','dxdy (P=5)']:
    #for q in ['efficiency [\% bw]','lat s [us]','lat d [us]']:
    for q in ['bandwidth [GB/s]','lat 1 [$\mu$s]','lat 4 [$\mu$s]']:
        tuples.append((p,q))
tuples[0] = ('axpby','bandwidth [GB/s]')
    

cols=pd.MultiIndex.from_tuples(tuples)

toDisk = pd.DataFrame(lines, index=index, columns=cols)
#toDisk.insert(0,('theo','[GB/s]'),theo)
filename='axpby-dot.tex'
with open(filename, 'wb') as f:
    f.write(bytes(toDisk.iloc[:,0:6].to_latex(escape=False,
                                              column_format='lp{1.7cm}p{0.75cm}p{0.75cm}p{1.7cm}p{0.75cm}p{0.75cm}',
                                              bold_rows=True),
                                              'UTF-8'))
toDisk.iloc[:,0:6]

axpby                                            dot  \
        bandwidth [GB/s] lat 1 [$\mu$s] lat 4 [$\mu$s] bandwidth [GB/s]   
i5           30 $\pm$ 01             00            n/a      10 $\pm$ 01   
skl         207 $\pm$ 06             00             00     193 $\pm$ 19   
knl         394 $\pm$ 23             06             10     142 $\pm$ 07   
gtx1060     158 $\pm$ 01             00            n/a      27 $\pm$ 01   
p100        554 $\pm$ 01             01             03     347 $\pm$ 02   
v100        849 $\pm$ 01             02             03     594 $\pm$ 03   

                                       
        lat 1 [$\mu$s] lat 4 [$\mu$s]  
i5                  05            n/a  
skl                 18             38  
knl                 55            120  
gtx1060             93            n/a  
p100                49             49  
v100                34             35

In [11]:
dxdy = toDisk.loc[:,[('dxdy (P=2)','bandwidth [GB/s]'),
                     ('dxdy (P=3)','bandwidth [GB/s]'),
                     ('dxdy (P=4)','bandwidth [GB/s]'),
                     ('dxdy (P=5)','bandwidth [GB/s]'),
                     ('dxdy (P=2)','lat 1 [$\mu$s]'),
                     ('dxdy (P=2)','lat 4 [$\mu$s]'),
                    ]]

dxdy.columns.set_levels(['dxdy (P=2)','dxdy (P=3)','dxdy (P=4)','dxdy (P=5)','dxdy','dxdy'],level=0,inplace=True)
dxdy.columns.set_labels([0,1,2,3,4,4],level=0,inplace=True)
filename='dxdy.tex'
with open(filename, 'wb') as f:
    f.write(bytes(dxdy.to_latex(escape=False,column_format='lp{1.9cm}p{1.9cm}p{1.9cm}p{1.9cm}p{0.75cm}p{0.75cm}',
                               bold_rows=True),'UTF-8'))
dxdy

dxdy (P=2)       dxdy (P=3)       dxdy (P=4)       dxdy (P=5)  \
        bandwidth [GB/s] bandwidth [GB/s] bandwidth [GB/s] bandwidth [GB/s]   
i5           28 $\pm$ 03      30 $\pm$ 03      26 $\pm$ 02      22 $\pm$ 02   
skl         182 $\pm$ 36     162 $\pm$ 13     119 $\pm$ 19     111 $\pm$ 09   
knl         240 $\pm$ 18     173 $\pm$ 27     127 $\pm$ 19     102 $\pm$ 15   
gtx1060     131 $\pm$ 01     112 $\pm$ 02      84 $\pm$ 14      70 $\pm$ 18   
p100        288 $\pm$ 03     238 $\pm$ 04     201 $\pm$ 02     166 $\pm$ 15   
v100        802 $\pm$ 17     713 $\pm$ 20     650 $\pm$ 16     536 $\pm$ 49   

                  dxdy                 
        lat 1 [$\mu$s] lat 4 [$\mu$s]  
i5                  00            n/a  
skl                 23             29  
knl                 10             53  
gtx1060             00            n/a  
p100                02             64  
v100                04             67